First, we create our camera class like this.  Please note, you can only create one USBCamera instance.

In [1]:
from jetcam.usb_camera import USBCamera
import ipywidgets, time
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from IPython.display import clear_output


In [2]:
display_as = "ipywidgets" # ipywidgets opencv

DISPLAY_WIDTH = 640
DISPLAY_HEIGHT = 480
CAMERA_WIDTH = 640
CAMERA_HEIGHT = 480

In [3]:
camera = USBCamera(width=DISPLAY_WIDTH, height=DISPLAY_HEIGHT, capture_width=CAMERA_WIDTH, 
                   capture_height=CAMERA_HEIGHT, capture_device=0)

We can then capture a frame from the camera like this

In [4]:
image = camera.read()
print("image.shape", image.shape)
print("camera.value.shape", camera.value.shape)

image.shape (480, 640, 3)
camera.value.shape (480, 640, 3)


Calling ``read`` also updates the camera's internal ``value``

In [5]:
class DummyTask:
    def __init__(self, value):
        self.value = value
    def ready(self):
        return True
    def get(self):
        return self.value
    
class Frames:
    def __init__(self):
        self.frames_count = 0
        self._time = time.time()
        self._old_fps = 0
    @property
    def fps(self):
        if int(time.time())%5 == 0 and self.frames_count > 0:
            result = self.frames_count/(time.time() - self._time)
#             print(self.frames_count, time.time() - self._time, result)
            self._old_fps = result
            self._time = time.time()
            self.frames_count = 0
        else:
            result = self._old_fps
        return result
    @property
    def fps_str(self):
        if self.fps > 0:
            return f"{self.fps:0.2f}"
        else: 
            return None
    def increment_frame(self):
        self.frames_count += 1
    def reset_frames_count(self):
        self.frames_count = 0
    
# frames = Frames()
# frames.increment_frame()
# frames.frames_count, frames.fps_str # frames.fps, 
    

You can create a widget to display this image.  You'll need to convert from bgr8 format to jpeg to stream to browser

In [6]:
if display_as == "ipywidgets":
    image_widget = ipywidgets.Image(format='jpeg')
    fps_widget = ipywidgets.Text("0")
    image_widget.value = bgr8_to_jpeg(image)
    display(image_widget)
    display(fps_widget)
elif display_as == "opencv":
    image_widget = DummyTask(image)
    fps_widget = None
    
if display_as == "ipywidgets":
    clear_output()
    

You can set the ``running`` value of the camera to continuously update the value in background.  This allows you to attach callbacks to the camera value changes. 

In [7]:
camera.running = True

frames = Frames()
def update_image(change):
    image = change['new']
    frames.increment_frame()
    if display_as == "ipywidgets":
        image_widget.value = bgr8_to_jpeg(image)
        if frames.frames_count > 150:
            fps_str = frames.fps_str
            if fps_str is not None:
                fps_widget.value = fps_str
    elif display_as == "opencv":
        image_widget.value = image


camera.observe(update_image, names='value')

if display_as == "ipywidgets":
    display(fps_widget)
    display(image_widget)

input("camera.unobserve")
if display_as == "ipywidgets":
    clear_output()
camera.unobserve(update_image, names='value')


Text(value='0')

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

KeyboardInterrupt: Interrupted by user

You can also use the traitlets ``dlink`` method to connect the camera to the widget, using a transform inbetween

In [ ]:
import traitlets

if display_as == "ipywidgets":
    camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
    display(image_widget)
elif display_as == "opencv":
    camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'))


You can remove this link like this

In [ ]:
input("camera_link.unlink")
camera_link.unlink()
# camera_link.link()